In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#Loading data

In [2]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/STAT 190/Data/new_data_dmarc.csv")
data['servedDate'] = pd.to_datetime(data['servedDate'], errors='coerce')
data['dob'] = pd.to_datetime(data['dob'], errors='coerce')
data.fillna("None", inplace=True)
db = data.copy()
db = db.drop([ 'primaryKey','locationLat','locationLng','middleName'], axis=1)
db.head(1)

<ipython-input-2-acaf7f4a4de1>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/drive/MyDrive/STAT 190/Data/new_data_dmarc.csv")
<ipython-input-2-acaf7f4a4de1>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  data.fillna("None", inplace=True)


,clientId,houseHoldIdAfn,dob,gender,race,education,foodstamps,dietaryIssue,veteran,service,servedDate,location,fiscalYear,householdMembers,fedPovertyLevel,annualIncome,incomeSource,category
0,CID-237272,HD-023070,1978-09-09 00:00:00,Female,Asian,Unknown,Yes,Unknown,No,Food Pantry,2019-01-04 06:00:00+00:00,Families Forward Bidwell Pantry,2019,4,125,31200.0,Not Employed,Stay at Home Parent/Caregiver


# Cleaning columns

In [4]:
#new
import pandas as pd

# 1) Subset to year 2024
data_2024 = data[data['servedDate'].dt.year == 2024]

# 2) Base aggregation by household
afn_data = data_2024.groupby("houseHoldIdAfn").agg({
    "annualIncome":      "max",
    "fedPovertyLevel":   "min",
    "gender":            "nunique",
    "race":              "nunique",
    "category":          "nunique",
    "education":         "first",
    "foodstamps":        "first",
    "dietaryIssue":      "first",
    "veteran":           "first",
    "service":           "first",
    "location":          "first",
    "householdMembers":  "first"
}).reset_index()

# 3) Count college‐degree holders per household
counts = []
for hh, group in data_2024.groupby('houseHoldIdAfn'):
    # filter within the loop to avoid a separate mask
    deg_group = group[group['education'].isin([
        'College Advanced Degree',
        'College 2 or 4 yr  Degree'
    ])]
    counts.append({
        'houseHoldIdAfn': hh,
        'education_count': deg_group['clientId'].nunique()
    })
education_counts = pd.DataFrame(counts)

# Merge in—this will work because afn_data has houseHoldIdAfn
afn_data = afn_data.merge(education_counts, on='houseHoldIdAfn', how='left')
afn_data['education_count'] = afn_data['education_count'].fillna(0).astype(int)

# 4) Race family (single vs mixed)
race_rows = []
for hh, group in data_2024.groupby('houseHoldIdAfn'):
    uniques = group['race'].unique()
    val = uniques[0] if len(uniques) == 1 else 'Mixed'
    race_rows.append({'houseHoldIdAfn': hh, 'race_family': val})
race_df = pd.DataFrame(race_rows)
afn_data = afn_data.merge(race_df, on='houseHoldIdAfn', how='left')

# 5) Category family
cat_rows = []
for hh, group in data_2024.groupby('houseHoldIdAfn'):
    uniques = group['category'].unique()
    val = uniques[0] if len(uniques) == 1 else 'Mixed'
    cat_rows.append({'houseHoldIdAfn': hh, 'category_family': val})
cat_df = pd.DataFrame(cat_rows)
afn_data = afn_data.merge(cat_df, on='houseHoldIdAfn', how='left')


# 7) Percent male
male_rows = []
for hh, group in data_2024.groupby('houseHoldIdAfn'):
    count_males = group[group['gender']=='Man (boy)']['clientId'].nunique()
    male_rows.append({'houseHoldIdAfn': hh, 'male_count': count_males})
male_df = pd.DataFrame(male_rows)
afn_data = afn_data.merge(male_df, on='houseHoldIdAfn', how='left')
afn_data['male_count'] = afn_data['male_count'].fillna(0)
afn_data['percent_male'] = afn_data['male_count'] / afn_data['householdMembers']

# 8) Drop per-client & unused columns
afn_data = afn_data.drop(columns=[
    'race','category','gender','male_count',
    'veteran','foodstamps','dietaryIssue'
])

# 9) Filter incomes
mask = (afn_data['annualIncome'] >= 0) & (afn_data['annualIncome'] <= 87063.55)
df = afn_data[mask].copy()

# 10) Income bins
bins = [0,1, 14000, 25800, 87001]
labels = ["0",'1-14,000','14,000-25,800','25,800-87,000']
df['income_bin'] = pd.cut(df['annualIncome'], bins=bins, labels=labels, right=False)

# 11) Months visited
data_2024['month'] = data_2024['servedDate'].dt.month
visit_list = []
for hh_month, group in data_2024.groupby(['houseHoldIdAfn','month']):
    hh, m = hh_month
    visit_list.append({'houseHoldIdAfn': hh, 'month': m})
visits_df = pd.DataFrame(visit_list)
months_rows = []
for hh, group in visits_df.groupby('houseHoldIdAfn'):
    months_rows.append({
        'houseHoldIdAfn': hh,
        'months_visited': group['month'].nunique()
    })
months_df = pd.DataFrame(months_rows)
df = df.merge(months_df, on='houseHoldIdAfn', how='left')
df['months_visited'] = df['months_visited'].fillna(0).astype(int)

# 12) Target flags
df['12_visits'] = 0
df['>6_visits'] = 0
for idx, row in df.iterrows():
    if row['months_visited'] == 12:
        df.at[idx, '12_visits'] = 1
    if row['months_visited'] >= 6:
        df.at[idx, '>6_visits'] = 1

# 13) Family-level flags

# Veteran
family_vet = {}
for hh, group in data_2024.groupby('houseHoldIdAfn'):
    vals = group['veteran'].unique()
    family_vet[hh] = vals[0] if len(vals)==1 else 'Unknown'

# Foodstamps
family_food = {}
for hh, group in data_2024.groupby('houseHoldIdAfn'):
    vals = group['foodstamps'].unique()
    family_food[hh] = vals[0] if len(vals)==1 else 'Unknown'

# Dietary issue
family_diet = {}
for hh, group in data_2024.groupby('houseHoldIdAfn'):
    has_issue = False
    for v in group['dietaryIssue']:
        if v not in ['None','None;None',]:
            has_issue = True
            break
    family_diet[hh] = has_issue

df['family_veteran']      = df['houseHoldIdAfn'].map(family_vet)
df['family_foodstamps']   = df['houseHoldIdAfn'].map(family_food)
df['family_dietaryIssue'] = df['houseHoldIdAfn'].map(family_diet)

df = df.drop(columns=['education','service'])

# Final cleaned DataFrame
cleaned = df
cleaned.head(1)


<ipython-input-4-d48e30b09b84>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2024['month'] = data_2024['servedDate'].dt.month


,houseHoldIdAfn,annualIncome,fedPovertyLevel,location,householdMembers,education_count,race_family,category_family,percent_male,income_bin,months_visited,12_visits,>6_visits,family_veteran,family_foodstamps,family_dietaryIssue
0,02-021406,0.0,0,DMARC-ket Southside Food Pantry,4,0,White,Unemployed No Income,0.0,0,1,0,0,No,Yes,False


#Checking for NA values

In [5]:
import pandas as pd
na_counts = cleaned.isna().sum()
na_table = pd.DataFrame({'Column': na_counts.index, 'NA Count': na_counts.values})
na_table

,Column,NA Count
0,houseHoldIdAfn,0
1,annualIncome,0
2,fedPovertyLevel,0
3,location,0
4,householdMembers,0
5,education_count,0
6,race_family,0
7,category_family,0
8,percent_male,0
9,income_bin,0


# Exporting CSV

In [ ]:
cleaned.to_csv("/content/drive/MyDrive/STAT 190/Data/2024_5-5-2025.csv", index=False)